In [1]:
class TicTacToe:
    def __init__(self):
        self.N = 3
        self.map = [['E' for _ in range(self.N)] for _ in range(self.N)]    # E: 빈 공간(Empty)
        self.map_index_description = [h*self.N + w for h in range(self.N) for w in range(self.N)]
        self.player_types = ('X', 'O')  # 선공: X, 후공: O
        self.global_step = 0

        self.win_reward = 1.0
        self.defeat_reward = -1.0
        self.draw_reward = 0.0
        self.player_result = {'X': self.draw_reward, 'O': self.draw_reward}

        self.done = False

    def reset(self):
        self.map = [['E' for _ in range(self.N)] for _ in range(self.N)]
        self.global_step = 0
        self.player_result = {'X': self.draw_reward, 'O': self.draw_reward}
        self.done = False

        return self.map

    def step(self, action):
        action_coord_h, action_coord_w = self.transform_action(action)
        if self.global_step % 2 == 0:
            current_player_idx = 0
            other_player_idx = 1
        else:
            current_player_idx = 1
            other_player_idx = 0
        current_player_type = self.player_types[current_player_idx]
        other_player_type = self.player_types[other_player_idx]
        if self.map[action_coord_h][action_coord_w] == 'E':
            self.map[action_coord_h][action_coord_w] = current_player_type
            if self.is_win(current_player_type):    # 현재 플레이어 승리
                self.player_result[current_player_type] = self.win_reward
                self.player_result[other_player_type] = self.defeat_reward
                self.done = True
            elif self.is_full():    # 무승부
                self.done = True
            else:
                pass
        else:   # 현재 플레이어 패배
            self.player_result[current_player_type] = self.defeat_reward
            self.player_result[other_player_type] = self.win_reward
            self.done = True
        self.global_step += 1

        return self.map, self.player_result, self.done

    def transform_action(self, action):
        return divmod(action, self.N)

    def is_win(self, current_player_type):
        vertical_win = [True for _ in range(self.N)]
        horizontal_win = [True for _ in range(self.N)]
        diagonal_win = [True for _ in range(2)]
        for h in range(self.N):
            for w in range(self.N):
                # 가로, 세로
                if self.map[h][w] != current_player_type:
                    vertical_win[h] = False
                    horizontal_win[w] = False
                else:
                    pass
                # 왼 대각
                if h == w and self.map[h][w] != current_player_type:
                    diagonal_win[0] = False
                # 오른 대각
                rotated_w = abs(w - (self.N - 1))
                if h == rotated_w and self.map[h][w] != current_player_type:
                    diagonal_win[1] = False
        if any(vertical_win) or any(horizontal_win) or any(diagonal_win):
            return True
        else:
            return False

    def is_full(self):
        for h in range(self.N):
            for w in range(self.N):
                if self.map[h][w] == 'E':
                    return False
                else:
                    pass
        return True

    def print_description(self):
        print("** Initial NxN Tic-tac-toe Map **")
        self.print_current_map()

        print("** Action Indexes **")
        for idx, des in enumerate(self.map_index_description):
            print(des, end=' ')
            if (idx + 1) % self.N == 0:
                print('\n', end='')

    def print_current_map(self):
        for h in range(self.N):
            for w in range(self.N):
                print(self.map[h][w], end=' ')
            print('\n', end='')
        print()

    # Fill this function
    def match_prediction(self):
        def is_over(map_, current_player):
            vertical_win = [True for _ in range(self.N)]
            horizontal_win = [True for _ in range(self.N)]
            diagonal_win = [True for _ in range(2)]
            for h in range(self.N):
                for w in range(self.N):
                    # 가로, 세로
                    if map_[h][w] != current_player:
                        vertical_win[h] = False
                        horizontal_win[w] = False
                    # 왼 대각
                    if h == w and map_[h][w] != current_player:
                        diagonal_win[0] = False
                    # 오른 대각
                    rotated_w = abs(w - (self.N - 1))
                    if h == rotated_w and map_[h][w] != current_player:
                        diagonal_win[1] = False
            if any(vertical_win) or any(horizontal_win) or any(diagonal_win):
                return 1
            
            for h in range(self.N):
                for w in range(self.N):
                    if self.map[h][w] == 'E':
                        return None
            return 0

        def minimizer(map_, current_player_idx, alpha, beta):
            # 말단 노드인 경우
            over = is_over(map_, self.player_types[current_player_idx])
            if over is not None:
                return -over
            v = float('inf') # 최솟값을 찾기 위해 v를 무한대로 초기화
            for action in [x for x in self.map_index_description if map_[x // self.N][x % self.N] =='E']:
                map_[action // self.N][action % self.N] = self.player_types[current_player_idx]
                max_v = maximizer(map_, (current_player_idx + 1) % len(self.player_types), alpha, beta)
                v = min(v, max_v)
                map_[action // self.N][action % self.N] = 'E'
                if v < alpha: # 구한 값이 alpha보다 작다면
                    return v # 해당 값을 반환해 가지치기
                beta = min(beta, v) # beta값 갱신
            return v # 구한 v값 반환
        
        def maximizer(map_, current_player_idx, alpha, beta):
            # 말단 노드인 경우
            over = is_over(map_, self.player_types[current_player_idx])
            if over is not None:
                return over
            v = float('-inf') # 최댓값을 찾기 위해 v를 음의 무한대로 초기화
            for action in [x for x in self.map_index_description if map_[x // self.N][x % self.N] =='E']:
                map_[action // self.N][action % self.N] = self.player_types[current_player_idx]
                min_v = minimizer(map_, (current_player_idx + 1) % len(self.player_types), alpha, beta)
                v = max(v, min_v) # minimizer 호출 후 v값 갱신
                map_[action // self.N][action % self.N] = 'E'
                if v > beta: # 구한 값이 beta보다 크다면
                    return v # 해당 값을 반환해 가지치기
                alpha = max(v, alpha) # alpha값 갱신
            return v # 구한 v값 반환
        
        res = maximizer(self.map, self.global_step % 2, float('-inf'), float('inf'))
        if res == 1:
            print('예측:', self.player_types[self.global_step % len(self.player_types)], '의 승리')
        elif res == -1:
            print('예측:', self.player_types[(self.global_step + 1) % len(self.player_types)], '의 승리')
        else:
            print('예측:', '무승부')


if __name__ == '__main__':
    game = TicTacToe()
    game.print_description()

    game.reset()
    done = False
    while not done:
        print()

        # Do it.
        game.match_prediction()

        action = int(input('Select action please: '))
        if not(game.map_index_description[0] <= action <= game.map_index_description[-1]):
            done = True
            print("Error: You entered the wrong number.")
            continue
        _, player_result, done = game.step(action)
        game.print_current_map()
        if done:
            for player, result in player_result.items():
                if result == game.win_reward:
                    player_result[player] = 'win'
                elif result == game.defeat_reward:
                    player_result[player] = 'defeat'
                else:
                    player_result[player] = 'draw'
            print(player_result)


** Initial NxN Tic-tac-toe Map **
E E E 
E E E 
E E E 

** Action Indexes **
0 1 2 
3 4 5 
6 7 8 

예측: 무승부
Select action please: 0
X E E 
E E E 
E E E 


예측: 무승부
Select action please: 4
X E E 
E O E 
E E E 


예측: 무승부
Select action please: 2
X E X 
E O E 
E E E 


예측: 무승부
Select action please: 1
X O X 
E O E 
E E E 


예측: 무승부
Select action please: 6
X O X 
E O E 
X E E 


예측: O 의 승리
Select action please: 3
X O X 
O O E 
X E E 


예측: O 의 승리
Select action please: 8
X O X 
O O E 
X E X 


예측: O 의 승리
Select action please: 5
X O X 
O O O 
X E X 

{'X': 'defeat', 'O': 'win'}
